### ETL - FATO GRAO

In [11]:
### Funcao Carregar Dados fato_grao
def carregar_dados_fato_grao(driver, server, database,dados, tabela_destino):
    
    con = conexao(driver, server, database)
  
    query = f'''INSERT INTO {tabela_destino} ''' + '''(''' + ','.join(list(dados)) + ''')''' + ''' values(''' + str('?,' * dados.shape[1])[:-1] + ''')'''
    
    cursor = con.cursor()
    print("Query Feita")
    
    try:
        for index, row in dados.iterrows():
            cursor.execute(query, row.IdData, row.IdPedido, row.IdExame, row.IdMaterial, row.IdUnidade, row.ValorProducao, row.Qtd_Exame, row.Qtd_Material)
        con.commit()
        cursor.close()
    except:
        print('Erro na Inserção dos Dados')
        
    return

In [12]:
### FATO GRAO

query = """SELECT DataLaudo, -- PK
        IdPedido, -- PK
        IdExame, -- PK
        Material, -- PK
        IdUnidade, -- PK
        ValorProducao FROM DATASET_1 """

try:
    new_data = pd.read_sql(query, con)
    data = pd.read_sql("SELECT IdData, Data FROM dim_data",con)
    material = pd.read_sql("SELECT IdMaterial, Material FROM dim_mat",con)
except:
    print("Erro na Leitura dos Dados")

if len(new_data) == 0:
    print("Tabela Vazia")
else: 
    
    ### Id Data
    new_data.drop_duplicates(subset = ['DataLaudo','IdPedido','IdExame','Material','IdUnidade'],keep = 'first', inplace = True)
    new_data['DataLaudo'] = pd.to_datetime(new_data['DataLaudo'].astype(str),format = "%Y%m%d")
    new_data['DataLaudo'] = new_data['DataLaudo'].astype(str)
    data['Data'] = data['Data'].astype(str)
    new_data = pd.merge(new_data, data, left_on = 'DataLaudo', right_on = "Data",how = 'left')
    
    ### Id Material
    new_data = pd.merge(new_data, material, on = 'Material',how = 'left')
    
    ## Quantidade de Exame Realizado
    qtd_exame = pd.DataFrame(new_data.groupby(by = "IdPedido")['IdExame'].nunique()).reset_index()
    qtd_exame.columns = ['IdPedido','Qtd_Exame']
    new_data = pd.merge(new_data, qtd_exame, on = 'IdPedido',how = 'left')
    
    ## Quantidade de Material 
    qtd_material = pd.DataFrame(new_data.groupby(by = "IdPedido")['IdMaterial'].nunique()).reset_index()
    qtd_material.columns = ['IdPedido','Qtd_Material']
    new_data = pd.merge(new_data, qtd_material, on = 'IdPedido',how = 'left')
    
    new_data = new_data[['IdData','IdPedido','IdExame','IdMaterial','IdUnidade','ValorProducao','Qtd_Exame','Qtd_Material']]
    try:
        truncate_table(driver, server, database, 'fato_grao')
        print("Tabela Truncada")
    except:
        print("Erro no Truncamento da Tabela")
        
    carregar_dados_fato_grao(driver, server, database, new_data, 'fato_grao')
    print("Insersao Concluida")